<a href="https://colab.research.google.com/github/eeolga/article/blob/main/Metamodel_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download the Log records into Notebook.

Delete unnecessary rows of e-course login records.


Save the results to an intermediate file.



In [1]:
import pandas as pd

# Load the Excel file again to start fresh
df = pd.read_excel('/content/131-2deadline.xlsx')

# Remove rows where 'Description' column contains '2949'
df_filtered_rows = df[~df['Description'].astype(str).str.contains('2949', na=False)]

# Remove specified columns
columns_to_drop = ['Event context', 'Event name']
df_updated = df_filtered_rows.drop(columns=columns_to_drop, errors='ignore')

# Save the updated DataFrame to '1.xlsx'
df_updated.to_excel('1.xlsx', index=False)

print("Rows with '2949' in 'Description' and columns 'Event context' and 'Event name' have been removed. The updated file is saved as '1.xlsx'.")


Rows with '2949' in 'Description' and columns 'Event context' and 'Event name' have been removed. The updated file is saved as '1.xlsx'.


Student's and resourse's name replacing to codes.

Save the results to an intermediate file.

In [2]:
import pandas as pd
import re

# Load the '1.xlsx' file (which is the result of the previous step)
df = pd.read_excel('1.xlsx')

# Function to extract the first number from a string
def get_first_number(text):
    numbers = re.findall(r'\d+', str(text))
    return numbers[0] if numbers else None

# Function to extract the last number from a string
def get_last_number(text):
    numbers = re.findall(r'\d+', str(text))
    return numbers[-1] if numbers else None

# Apply the functions to update the specified columns
df['User full name'] = df['Description'].apply(get_first_number)
df['Component'] = df['Description'].apply(get_last_number)

# Save the updated DataFrame to '2.xlsx'
df.to_excel('2.xlsx', index=False)

print("The 'User full name' and 'Component' columns have been updated based on the 'Description' column. The file has been saved as '2.xlsx'.")

# Display the first few rows of the updated DataFrame to confirm
display(df.head())

The 'User full name' and 'Component' columns have been updated based on the 'Description' column. The file has been saved as '2.xlsx'.


,Time,User full name,Component,Description
0,"16/11/25, 23:42:18",32926,233297,The user with id '32926' has viewed the submis...
1,"16/11/25, 23:42:18",32926,233297,The user with id '32926' viewed the 'assign' a...
2,"16/11/25, 23:42:18",32926,233297,The user with id '32926' has submitted the sub...
3,"16/11/25, 23:42:18",32926,233297,The user with id '32926' has accepted the stat...
4,"16/11/25, 23:42:18",32926,233297,The user with id '32926' created a file submis...


Grouping logs by user.

Save the results to an intermediate file.

In [5]:
import pandas as pd

# Load the '2.xlsx' file
df = pd.read_excel('2.xlsx')

# Remove the 'Description' column
df_no_description = df.drop(columns=['Description'], errors='ignore')


# compute total events per user
user_totals = df_aggregated.groupby("User full name")["Count of Events"].sum().reset_index()

# Save the updated DataFrame to '3.xlsx'
df_aggregated.to_excel('3.xlsx', index=False)

# Display the first few rows of the aggregated DataFrame to confirm
display(df_aggregated.head())

display(user_totals.head()) # Also display user_totals

,User full name,Component,Count of Events
0,26512.0,6294.0,1
1,26512.0,233142.0,4
2,26512.0,233277.0,15
3,26512.0,233284.0,18
4,26512.0,233297.0,14


,User full name,Count of Events
0,26512.0,76
1,29213.0,704
2,32686.0,972
3,32760.0,142
4,32918.0,506


Calculating the resource usage time for a user.

Save the results to an intermediate file.

In [8]:
import pandas as pd

# Assuming user_totals DataFrame is already available from previous steps
# If not, you would load it from '3.xlsx':
# user_totals = pd.read_excel('3.xlsx')

# Define the constants for WTglobal calculation
# ∑resource = 3 * 26 academic hours
# 1 academic hour = 45 minutes = 45 * 60 seconds = 2700 seconds
sum_resource_academic_hours = 3 * 26
sum_resource_seconds = sum_resource_academic_hours * 45 * 60 # Convert to seconds

Ij = 0.14

# Calculate WTglobal for each user
# Tr = user_totals['Total Time (seconds)']
user_totals['WTglobal'] = (user_totals['Total Time (seconds)'] / sum_resource_seconds) * Ij

# Save the updated DataFrame to '4.xlsx'
user_totals.to_excel('6.xlsx', index=False)

print("WTglobal calculated and saved to '4.xlsx'.")

# Display the first few rows of the updated DataFrame
display(user_totals.head())

WTglobal calculated and saved to '6.xlsx'.


,User full name,Count of Events,Total Time (seconds),Total Time (minutes),Total Time (hours),WTglobal
0,26512.0,76,6384.0,106.4,1.773333,0.004244
1,29213.0,704,59136.0,985.6,16.426667,0.039312
2,32686.0,972,81648.0,1360.8,22.680000,0.054277
3,32760.0,142,11928.0,198.8,3.313333,0.007929
4,32918.0,506,42504.0,708.4,11.806667,0.028255


Calculating WTglobal for each user.

Save the results to an intermediate file.

In [7]:
import pandas as pd

# Assuming user_totals DataFrame is already available from previous steps
# If not, you would load it from '4.xlsx':
# user_totals = pd.read_excel('4.xlsx')

# Define the constants from the formula
pa = 0.7
i_bar = 120 # seconds

# Calculate seconds per event
seconds_per_event = pa * i_bar

# Calculate 'Total Time (seconds)' per user
user_totals['Total Time (seconds)'] = user_totals['Count of Events'] * seconds_per_event

# Optionally, convert to minutes and hours for readability
user_totals['Total Time (minutes)'] = user_totals['Total Time (seconds)'] / 60
user_totals['Total Time (hours)'] = user_totals['Total Time (minutes)'] / 60

# Save the updated DataFrame to '5.xlsx'
user_totals.to_excel('5.xlsx', index=False)

print("Total time per user calculated and saved to '5.xlsx'.")

# Display the first few rows of the updated DataFrame
display(user_totals.head())

Total time per user calculated and saved to '5.xlsx'.


,User full name,Count of Events,Total Time (seconds),Total Time (minutes),Total Time (hours)
0,26512.0,76,6384.0,106.4,1.773333
1,29213.0,704,59136.0,985.6,16.426667
2,32686.0,972,81648.0,1360.8,22.680000
3,32760.0,142,11928.0,198.8,3.313333
4,32918.0,506,42504.0,708.4,11.806667
